In [1]:
from datetime import date

from create_data import recreate_databases, create_tables
from jobs.order_created import do
from utils import get_spark_session

In [2]:
spark = get_spark_session()

2021-11-24 17:31:16.859 | INFO     | utils:get_spark_session:7 - Creating Spark Session


In [3]:
recreate_databases(spark)

2021-11-24 17:31:20.638 | INFO     | create_data:recreate_databases:29 - Removing spark warehouse (path = 'spark-warehouse')
2021-11-24 17:31:20.672 | INFO     | create_data:recreate_databases:33 - Creating database 'standardized_glovo_live'
2021-11-24 17:31:21.854 | INFO     | create_data:recreate_databases:33 - Creating database 'mpcustomer_custom_events'
2021-11-24 17:31:21.874 | INFO     | create_data:recreate_databases:33 - Creating database 'mpcustomer_screen_views'
2021-11-24 17:31:21.895 | INFO     | create_data:recreate_databases:33 - Creating database 'enriched_custom_events'
2021-11-24 17:31:21.911 | INFO     | create_data:recreate_databases:33 - Creating database 'enriched_screen_views'


In [4]:
create_tables(spark)

2021-11-24 17:31:21.934 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.cities'
2021-11-24 17:31:26.126 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.devices'
2021-11-24 17:31:29.600 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.orders'
2021-11-24 17:31:33.070 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.customer_subscriptions'
2021-11-24 17:31:36.488 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.customer_subscription_periods'
2021-11-24 17:31:39.783 | INFO     | create_data:create_table:91 - Creating table 'mpcustomer_custom_events.order_created'


In [5]:
do(spark, exec_date=date(2021, 11, 19), n_days=3)

2021-11-24 17:31:43.553 | INFO     | loaders:load:30 - Loading 'mpcustomer_custom_events.order_created'
2021-11-24 17:31:43.630 | INFO     | loaders:load:30 - Loading 'standardized_glovo_live.cities'
2021-11-24 17:31:43.688 | INFO     | loaders:load:30 - Loading 'standardized_glovo_live.orders'
2021-11-24 17:31:43.783 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddTimezone'
2021-11-24 17:31:43.804 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddLocalTime'
2021-11-24 17:31:43.916 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddNumberOfOrders'
2021-11-24 17:31:44.078 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddIsPrime'
2021-11-24 17:31:44.131 | INFO     | writers:write:23 - Writting 'enriched_custom_events.order_created'


# Show input

In [6]:
spark.table("mpcustomer_custom_events.order_created").show()

+-----------------------+-----------+-------------------+---------------+
|custom_attributes__city|customer_id|      creation_time|p_creation_date|
+-----------------------+-----------+-------------------+---------------+
|                    BCN|          1|2021-11-19 12:00:00|     2021-11-19|
|                    BCN|          1|2021-11-18 08:00:00|     2021-11-18|
|                    CAG|          2|2021-11-17 05:05:00|     2021-11-17|
|                    CAG|          2|2021-11-01 07:29:00|     2021-11-01|
+-----------------------+-----------+-------------------+---------------+



# Show outputs

In [7]:
spark.table("enriched_custom_events.order_created").show()

+-----------------------+-----------+-------------------+---------------+-------------+------------+-------------------+-----------+----------------+--------+
|custom_attributes__city|customer_id|      creation_time|p_creation_date|    time_zone|country_code|creation_time_local|time_of_day|number_of_orders|is_prime|
+-----------------------+-----------+-------------------+---------------+-------------+------------+-------------------+-----------+----------------+--------+
|                    CAG|          2|2021-11-17 05:05:00|     2021-11-17|  Europe/Rome|          IT|2021-11-17 06:05:00|  Breakfast|               1|    true|
|                    BCN|          1|2021-11-19 12:00:00|     2021-11-19|Europe/Madrid|          ES|2021-11-19 13:00:00|      Lunch|               2|   false|
|                    BCN|          1|2021-11-18 08:00:00|     2021-11-18|Europe/Madrid|          ES|2021-11-18 09:00:00|  Breakfast|               2|   false|
+-----------------------+-----------+---------